In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from datasets import load_dataset,load_from_disk
from datasets import Dataset
from swanlab.integration.huggingface import SwanLabCallback
from transformers import DataCollatorForSeq2Seq, Trainer
import json
import pyarrow.parquet as pq
import numpy as np
import re
from tqdm import tqdm
import time

/home/vipuser/miniconda3/envs/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-31 12:45:38,707] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/vipuser/miniconda3/envs/openr1/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/vi

In [2]:
import pandas as pd
file_path = "../GRPO_data/training_prompt2.csv"
# model_path = "/home/LLM/models/Qwen2.5-0.5B-Instruct"
model_path = "../SFT_model"
df = pd.read_csv(file_path)
df = df.iloc[200:300]

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_path)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [4]:
print(df)

                   db_id                                           question  \
200         entrepreneur  What are the names of people in ascending orde...   
201  document_management  What is the total access count of documents th...   
202                  geo   what is the combined population of all 50 states   
203                  geo          how high is the highest point of delaware   
204              musical  Show names of actors and names of musicals the...   
..                   ...                                                ...   
295               game_1  What are the names of all the video games and ...   
296                 farm  Give the average number of working horses on f...   
297          city_record  Which cities' temperature in March is lower th...   
298         small_bank_1  Find the name and id of accounts whose checkin...   
299              music_2  What is the last name of the musician that has...   

                                                 qu

In [5]:
system_prompt = """
You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>\nYour Thought Process (Why were these choices made?)\n</think>\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> 
"""
print(system_prompt)


You are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. 
Follow this response template: <think>
Your Thought Process (Why were these choices made?)
</think>
<answer>
###Tables: (Related Table);
###Columns: (Related Columns from Tables);
</answer> 



In [6]:
def make_user_prompt(question, database_schema):
    return f"<question>{question}</question>\n<database>{database_schema}</database>"

In [7]:
def make_conversation(question, database_schema):
    conversation = []
    conversation0 = {}
    conversation1 = {}
    conversation2 = {}
    user_prompt = make_user_prompt(question, database_schema)
    conversation0["role"] = "system"
    conversation0["content"] = system_prompt
    conversation1["role"] = "user"
    conversation1["content"] = user_prompt + "\n"
    conversation.append(conversation0)
    conversation.append(conversation1)
#     print(conversation2["content"])
    return conversation

In [8]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=2048
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     print(response)
     
    return response

In [9]:
print(len(df))

100


In [10]:
sample_list = []
correct_list = []
for index, row in df.iterrows():
    correct_list.append(row["target_schema"])
    messages = make_conversation(row["question"],row["database_schema"])
    print(messages)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    response = predict(messages, model, tokenizer)
    sample_list.append(response)
    print(response)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<next>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    if index >= 210:
        break

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[{'role': 'system', 'content': "\nYou are a Schema linking assistant. Your task is to select appropriate table names and corresponding column names from the database in <database>...</database> based on the user's question in <question>...</question>. \nFollow this response template: <think>\nYour Thought Process (Why were these choices made?)\n</think>\n<answer>\n###Tables: (Related Table);\n###Columns: (Related Columns from Tables);\n</answer> \n"}, {'role': 'user', 'content': '<question>What are the names of people in ascending order of weight?</question>\n<database>CREATE TABLE `entrepreneur` (\n  Entrepreneur_ID INT PRIMARY KEY,\n  People_ID INT REFERENCES people(People_ID),\n  Company TEXT,\n  Money_Requested REAL,\n  Investor TEXT\n);\n\nCREATE TABLE `people` (\n  People_ID INT PRIMARY KEY,\n  Name TEXT,\n  Height REAL,\n  Weight REAL,\n  Date_of_Birth TEXT\n);\n\n</database>\n'}]
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<message_end>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

<think>
Okay, let me try to figure out how to answer the user's question: "What is the combined population of all 50 states?" First, I need to understand which tables and columns in the provided database schema would be relevant.

Looking at the database schema, there are several tables: state, city, border_info, highlow, lake, mountain, and river. The question asks for the combined population of all 50 states, so it probably involves population data from the state table.

The state table has columns like state_name, population, area, country_name, capital, and density. Since we want the combined population of all 50 states, each row in the state table should represent one state. So, joining with other tables might not directly relate to population, but maybe related to administrative divisions or borders.

Other tables like city, border_info, highlow, lake, mountain, and river don't seem directly involved here unless they're part of some larger query involving population calculations.

<think>
Okay, let me try to figure out how to approach this question. The user wants to know which rooms are available for members of faculty who are professors and live in Building NEB.

First, I need to identify which tables and columns are involved here. Looking at the database schema, there are three main tables: Student, Faculty, and Member_of. Let me check those ones one by one.

The Student table has columns like StuID, Lname, Fname, Age, Sex, Major, Advisor, and city_code. That seems directly related to the question because it's about students' details and their affiliation with faculty. 

The Faculty table includes Faculty_ID, Lname, Fname, Rank, Sex, Phone, Room, Building, and DPhone. It's important that we connect this Faculty table to the Student table through FacID. So each entry in Faculty would correspond to a student whose advisor is also a professor. 

The Member_of table links Faculty to DNO via FacID. Each entry in Member_of represents a student living in Building NE

<think>
Okay, let me try to figure out how to answer the user's question: "What is the average price range of hotels for each each star rating code?" 

First, I need to look at the database schema provided. There are three tables: Ref_Hotel_Star_Ratings, Locations, and others. The question mentions stars and ratings codes, so those two tables are definitely relevant.

Looking at the Ref_Hotel_Star_Ratings table, it has columns like star_rating_code and star_rating_description. That confirms that we're dealing with star ratings. Then there's the Locations table which has details about where the hotels are located. The Hotels table links to Locations via star_rating_code, but the question specifically asks for star ratings per star code. So maybe we don't need all the data from Locations unless we're joining with another table. But since the question only asks by star code, maybe just the star rating code is needed. Wait, wait, the star rating code itself might be part of the location de

<think>
Okay, let me try to figure out how to answer the question. The user wants to list the clubs where "Davis Steven" is a member. First, I need to identify which tables and columns are relevant here.

Looking at the database schema, there are three main tables: Student, Club, and Member_of_club. The question mentions "club" and "member", so probably Club or Club and Member_of_club. The student table has information about students like their ID, last name, first name, age, etc., but not details about club memberships. 

The Club table has Club Name, Description, Location, and other details. Since the question is about clubs with a specific member, maybe we should start by looking for members who have Davis Steven listed. But wait, the Club table also has a ClubID which might be unique for each club. So joining with Club would involve selecting Club.Name and ClubID from Club that match Davis Steven's name. Then, filtering those results to get only those with a ClubID associated with 

In [11]:
#format_rule_base
print(len(sample_list))
print(len(correct_list))
print(correct_list[1])

11
11
###Tables: documents;
###Columns: documents.access_count, documents.document_type_code;



In [12]:
def check_socore_label(ground_truth ,answer):
#     keys_same = set(dict1.keys()) == set(dict2.keys())
    reward = 0     
    # set max accept score
    tabel_reward = 10
    columns_reward = 8
    
    # set max penalty 
    table_penalty = 4
    columns_penalty= 2
    
    # get ground truth      
    ground_truth_tables_match = re.search(r'Tables:\s*(.*?);', ground_truth)
    ground_truth_columns_match = re.search(r'Columns:\s*(.*?);', ground_truth)
    
    truth_tables = set(ground_truth_tables_match.group(1).split(', ')) if ground_truth_tables_match else set()
    truth_columns = set([col.strip() for col in ground_truth_columns_match.group(1).split(', ')]) if ground_truth_columns_match else set()
    
    
    tables_match =  re.search(r'Tables:\s*(.*?);', answer)
    columns_match = re.search(r'Columns:\s*(.*?);', answer)
    pre_tables = set(tables_match.group(1).split(', ')) if tables_match else set()
    pre_columns = set([col.strip() for col in columns_match.group(1).split(', ')]) if columns_match else set()
    
    
    # reward &  penalty   
    avg_table_format_score = tabel_reward / len(truth_tables)
    avg_column_format_score = columns_reward / len(truth_columns)
    
    if len(pre_tables) != 0:
        avg_table_fail_penalty = table_penalty / len(pre_tables)
    if len(pre_columns) != 0:
        avg_column_fail_penalty = columns_penalty / len(pre_columns)
    
    
    table_format_len = len(truth_tables & pre_tables)
    column_format_len = len(truth_columns & pre_columns)
    
    
    table_format_reward =  table_format_len * avg_table_format_score
    column_format_reward =  column_format_len * avg_column_format_score
    
    reward += table_format_reward
    reward += column_format_reward
    
#     print(table_format_len)
#     print(truth_tables)
    table_fail_penalty = 0
    column_fail_penalty = 0
    
    if table_format_len == 0:
        #  missing all table
        reward -= 6
        print("missing all table")
    else:
        table_fail_len = len(pre_tables - truth_tables)
        table_fail_penalty = table_fail_len * avg_table_fail_penalty
        reward -= table_fail_penalty
        
    if column_format_len == 0:
        #  missing all columns
        reward -= 4
        print("missing all column")
    else:
        column_fail_len = len(pre_columns - truth_columns)
        column_fail_penalty = column_fail_len * avg_column_fail_penalty
        reward -= column_fail_penalty
    print("table_format_reward:", table_format_reward)
    print("column_format_reward:", column_format_reward)
    print("table_fail_penalty:", table_fail_penalty)
    print("column_fail_penalty:", column_fail_penalty)
    
    print("final_reward",reward)
    return reward            

In [13]:
def format_reward(completions, **kwargs):
    pattern = r"^<think>\n.*?\n</think>\n<answer>\n.*?\n</answer>$"
    matches = re.match(pattern, completions, re.DOTALL | re.MULTILINE)
    print(1.0 if matches else 0.0)
    return 1.0 if matches else 0.0


def tag_count_reward(completions, **kwargs) -> list[float]:
    count = 0
    if completions.count("<think>\n") == 1:
        count += 0.25
    if completions.count("\n</think>\n") == 1:
        count += 0.25
    if completions.count("\n<answer>\n") == 1:
        count += 0.25
    if completions.count("\n</answer>") == 1:
        count += 0.25
    if completions.count("###Tables:") == 1:
        count += 0.25
    if completions.count("###Columns") == 1:
        count += 0.25
    print(count)
    return count
    

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
def gen_len_reward(completions, **kwargs):
    reward = 0
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    complete_token = tokenizer(completions)
    complete_token_len = len(complete_token["input_ids"])
    print(complete_token_len)
    if 0 <complete_token_len < 500:
        reward += 0.5
    elif 500 <=complete_token_len < 800:
        reward += 1
    elif 800 <=complete_token_len < 1200:
        reward += 0
    elif 1200 <= complete_token_len < 1800:
        reward -= 2
    elif 1800 <= complete_token_len:
        reward -= 4
    print("len_reward",reward)
    print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    return reward
        
    

In [15]:
for index in range(len(sample_list)):
#     print(correct_list[index])
#     print(sample_list[index])
#     print("++++++++++++++++++++++++++++++++")
#     break
    check_socore_label(correct_list[index],sample_list[index])
    format_reward(sample_list[index])
    tag_count_reward(sample_list[index])
    print("+++++++++++++++++++++++++++")
#     gen_len_reward(sample_list[index])

missing all column
table_format_reward: 10.0
column_format_reward: 0.0
table_fail_penalty: 0.0
column_fail_penalty: 0
final_reward 6.0
1.0
1.5
+++++++++++++++++++++++++++
missing all table
missing all column
table_format_reward: 0.0
column_format_reward: 0.0
table_fail_penalty: 0
column_fail_penalty: 0
final_reward -10.0
1.0
1.5
+++++++++++++++++++++++++++
table_format_reward: 10.0
column_format_reward: 8.0
table_fail_penalty: 0.0
column_fail_penalty: 1.0
final_reward 17.0
1.0
1.5
+++++++++++++++++++++++++++
missing all table
missing all column
table_format_reward: 0.0
column_format_reward: 0.0
table_fail_penalty: 0
column_fail_penalty: 0
final_reward -10.0
1.0
1.5
+++++++++++++++++++++++++++
missing all column
table_format_reward: 5.0
column_format_reward: 0.0
table_fail_penalty: 0.0
column_fail_penalty: 0
final_reward 1.0
1.0
1.5
+++++++++++++++++++++++++++
missing all column
table_format_reward: 10.0
column_format_reward: 0.0
table_fail_penalty: 2.0
column_fail_penalty: 0
final_rewa

In [16]:
import re
 
text = """
###Tables: documents;
###Columns: documents.access_count, documents.document_type_code;
"""
 
# 匹配 Tables 部分
tables_match = re.search(r'Tables:\s*(.*?);', text)
tables = tables_match.group(1).split(', ') if tables_match else []
 
# 匹配 Columns 部分
columns_match = re.search(r'Columns:\s*(.*?);', text)
columns = [col.strip() for col in columns_match.group(1).split(', ')] if columns_match else []
 
print("Tables:", tables)
print("Columns:", columns)

Tables: ['documents']
Columns: ['documents.access_count', 'documents.document_type_code']


In [17]:
A = set(["a","b","c","e"])
B = set(["a","b","d"])
print(A-(A-B))

{'b', 'a'}
